In [1]:
import os
curPath = os.getcwd()+os.sep
import numpy as np

In [2]:
#问答句文件中正样本生负样本
def getSamples(fileName):
    #默认是认为文件中基本都是正样本，可以存在少数负样本，但是正样本肯定要比负样本多很多，所以才需要生成一些负样本
    question_answer,label = [],[]
    correctQA = []
    assert os.path.isfile(curPath+fileName)
    print(curPath+fileName)
    with open(curPath+fileName,"r+",encoding="utf-8") as file:
        for line in file:
            contents = line.strip().split()
            if len(contents)<2:
                continue
            question_answer.append(contents[0]+"  "+contents[1])
            if contents[2].lower() == 'y':
                correctQA.append(contents[0]+"  "+contents[1])
                label.append(1)
            else:
                label.append(0)
    for i in range(len(correctQA)-(len(question_answer)-len(correctQA))):
        #即统计正确的个数-错误的个数，目的是为了最终让正样本和负样本比例1:1
        wrong_num = np.random.randint(0,len(correctQA))
        while wrong_num == i:
            wrong_num = np.random.randint(0,len(correctQA))
        question_answer.append(correctQA[i].strip().split()[0]+"  "+correctQA[wrong_num].strip().split()[1])
        label.append(0)
    return question_answer,label

In [3]:
fileName = "QA.txt"
question_answer,label = getSamples(fileName)
print("question_answer:",question_answer)
print("label:",label)

D:\Anaconda\workSpace\深度学习\QA.txt
question_answer: ['玫瑰花是什么颜色的  红色的', '向日葵是什么颜色的  黄色的', '鲫鱼生活在哪里  水里', '啄木鸟生活在哪里  树上', '太阳从哪边升起  东方', '太阳从哪边落下  西边', '霖淼书生是谁  周周', '霖淼书生是哪所大学毕业的  北邮', '霖淼书生现在在哪个城市  北京', '霖淼书生老家在哪个城市  浙江', '玫瑰花是什么颜色的  北京', '向日葵是什么颜色的  北邮', '鲫鱼生活在哪里  西边', '啄木鸟生活在哪里  黄色的', '太阳从哪边升起  树上', '太阳从哪边落下  浙江', '霖淼书生是谁  水里', '霖淼书生是哪所大学毕业的  黄色的', '霖淼书生现在在哪个城市  北邮', '霖淼书生老家在哪个城市  北京']
label: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
#word2vec中随机负采样，将不等距映射到等距上
from collections import Counter
import re
import numpy as np
def negativeSample(trainData,number=1,M=1e+6):
    #number代表窗口，即一个正样本需要多少个负样本,M为等间距对[0,1]拆分
    #返回trainData，negativeData
    trainData = re.sub("\s+","",trainData)
    word_dict = Counter(trainData)
    length = len(trainData)
    word_dict = {key:value/length for key,value in word_dict.items()}
    #print(word_dict)
    
    positive = []
    negative = []

    for word in trainData:
        for idx in range(number):
            positive.append(word)
            negative_word = word
            while negative_word == word:
                random_num = np.random.randint(0,M)/M
                negative_word = get_negative(word_dict,random_num)
            negative.append(negative_word)
    
    return positive,negative

def get_negative(word_dict,random_num):
    total_length = 0
    for word,length in word_dict.items():
        total_length += length
        if total_length >= random_num:
            return word
        

In [5]:
trainData = "ri chu jiang hua hong sheng huo chun lai jiang shui lv ru lan"
#trainData = "春花秋月何时了"*10+"往事知多少"*5
positive,negative = negativeSample(trainData,number=2,M=100)
print(positive,"\n","="*20)
print(negative)

['r', 'r', 'i', 'i', 'c', 'c', 'h', 'h', 'u', 'u', 'j', 'j', 'i', 'i', 'a', 'a', 'n', 'n', 'g', 'g', 'h', 'h', 'u', 'u', 'a', 'a', 'h', 'h', 'o', 'o', 'n', 'n', 'g', 'g', 's', 's', 'h', 'h', 'e', 'e', 'n', 'n', 'g', 'g', 'h', 'h', 'u', 'u', 'o', 'o', 'c', 'c', 'h', 'h', 'u', 'u', 'n', 'n', 'l', 'l', 'a', 'a', 'i', 'i', 'j', 'j', 'i', 'i', 'a', 'a', 'n', 'n', 'g', 'g', 's', 's', 'h', 'h', 'u', 'u', 'i', 'i', 'l', 'l', 'v', 'v', 'r', 'r', 'u', 'u', 'l', 'l', 'a', 'a', 'n', 'n'] 
['a', 'h', 'v', 'g', 'i', 'h', 'o', 'l', 'n', 'o', 'g', 'l', 'g', 'h', 'v', 'l', 'u', 'g', 'h', 'i', 's', 'n', 'o', 's', 'n', 'r', 'a', 'v', 'l', 'a', 'a', 'i', 'c', 'j', 'j', 'h', 'u', 'n', 'n', 'i', 'g', 'i', 's', 'l', 'g', 'j', 'n', 'n', 'u', 'u', 'a', 'h', 'l', 'g', 'h', 'h', 'i', 'l', 'j', 'n', 'n', 'v', 'n', 'n', 'u', 'g', 'n', 'c', 'o', 'n', 'v', 'h', 'n', 'c', 'l', 'h', 'g', 'a', 'g', 'h', 'a', 'e', 'r', 's', 'i', 'e', 'u', 'v', 'r', 'h', 'j', 'i', 'h', 'h', 'j', 'i']
